## Desafio Semana 3 - Data Science Codenation

### Objetivo

Trabalhando com uma base de consumidores, queremos conhecer melhor nossos clientes por estado. Para isso, iniciamos uma análise na pontuação de crédito, e vamos responder a pergunta: qual estado possui os clientes com melhores pontuações de crédito?

### Set up da análise

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('desafio1.csv')

### Iniciando a análise

In [3]:
df.head()

RowNumber                                        id  \
0          1  e7f44fcbd380d4cef7e6c232cc7e37895c3fd197   
1          2  28dcb083ad90512da16b9430085c2cddb8ca5e12   
2          3  774bc378f787438c9c7594e536787d07a097a54b   
3          4  043a71326f7096de155e7f0c559dc62b5e4b7239   
4          5  37b3bb8ca243955fb3605ec7e1c2a4607cdb3b22   

                                  sobrenome  pontuacao_credito  \
0  6d6e0aa1b9b413e442e2fb68df14b4fc3f91de50                619   
1  48e1ad846796fa314f1b4a6702b83343eb5482c5                608   
2  f2b29d5d934de615812b697132e767dea0f1e9e2                502   
3  85d5c9da7cddd8109ad32a6c348fe2bb10bf99f2                699   
4  ddc89c837a6933639de75f28171057060bd322df                850   

  estado_residencia genero  idade  nivel_estabilidade  saldo_conta  \
0                SC      F     42                   2         0.00   
1                RS      F     41                   1     83807.86   
2                SC      F     42                   8    159660.80   
3                SC      F     39                   1         0.00   
4                RS      F     43                   2    125510.82   

   numero_produtos  possui_cartao_de_credito  membro_ativo  
0                1                         1             1  
1                1                         0             1  
2                3                         1             0  
3                2                         0             0  
4                1                         1             1

In [4]:
# Exibindo o tipo de cada variável
df.dtypes

RowNumber                     int64
id                           object
sobrenome                    object
pontuacao_credito             int64
estado_residencia            object
genero                       object
idade                         int64
nivel_estabilidade            int64
saldo_conta                 float64
numero_produtos               int64
possui_cartao_de_credito      int64
membro_ativo                  int64
dtype: object

In [5]:
# Conferindo se tem dados faltantes
aux = pd.DataFrame({'colunas': df.columns,
                    'tipos': df.dtypes,
                    'percentual_faltante': df.isnull().sum() / df.shape[0]})
aux

colunas    tipos  \
RowNumber                                RowNumber    int64   
id                                              id   object   
sobrenome                                sobrenome   object   
pontuacao_credito                pontuacao_credito    int64   
estado_residencia                estado_residencia   object   
genero                                      genero   object   
idade                                        idade    int64   
nivel_estabilidade              nivel_estabilidade    int64   
saldo_conta                            saldo_conta  float64   
numero_produtos                    numero_produtos    int64   
possui_cartao_de_credito  possui_cartao_de_credito    int64   
membro_ativo                          membro_ativo    int64   

                          percentual_faltante  
RowNumber                                 0.0  
id                                        0.0  
sobrenome                                 0.0  
pontuacao_credito                         0.0  
estado_residencia                         0.0  
genero                                    0.0  
idade                                     0.0  
nivel_estabilidade                        0.0  
saldo_conta                               0.0  
numero_produtos                           0.0  
possui_cartao_de_credito                  0.0  
membro_ativo                              0.0

In [6]:
# Análise estatístic univariada da coluna 'pontuacao_credito'
df['pontuacao_credito'].describe()

count    7000.000000
mean      649.787000
std        97.046279
min       350.000000
25%       582.000000
50%       651.000000
75%       717.000000
max       850.000000
Name: pontuacao_credito, dtype: float64

In [7]:
# Skewness - Negativa
df.pontuacao_credito.skew()

-0.046834946542205005

In [8]:
# Kurtose - Platicurtica
df.pontuacao_credito.kurtosis()

-0.4629012469890421

### Analisando por Estado

In [9]:
# Quais estados estão contemplados na base de dados?

df['estado_residencia'].unique()

array(['SC', 'RS', 'PR'], dtype=object)

In [10]:
# Calculando a média, mediana e desvio padrão por estado. 
df_resul = df.groupby('estado_residencia').agg({'pontuacao_credito': ['mean', 'median', 'std']})
df_resul.columns = ['mean_pontuacao_credito', 'median_pontuacao_credito', 'std_pontuacao_credito']
df_resul = pd.DataFrame(df_resul)
df_resul = df_resul.reset_index()

In [11]:
# Calculando a moda por estado.
# A função moda não é agregadora, então não se aplica diretamente ao groupby. Para obter a moda é necessário fazer o seguinte:
aux2 = df.groupby('estado_residencia')['pontuacao_credito'].apply(lambda x: x.mode())
aux2 = pd.DataFrame(aux2)
aux2 = aux2.reset_index()

In [12]:
# Agregando as informações, e mostrando qual a média, mediana, moda e desvio padrão da pontuação de crédito por estado
df_resul['mode_pontuacao_credito'] = aux2['pontuacao_credito']

# Renomeando colunas
df_resul.columns = ['estado', 'media', 'mediana', 'desvio_padrao', 'moda']

# Mudando o tipo da variável
df_resul['moda'] = df_resul.moda.astype(float)
df_resul['mediana'] = df_resul.mediana.astype(float)

# Reordenando colunas
df_resul = df_resul[['estado','media', 'mediana','moda','desvio_padrao']]

In [13]:
# Definindo o estado como index
df_resul.set_index('estado', inplace=True)

### Solução

In [14]:
# Qual a média, mediana, moda e desvio padrão da pontuação de crédito para cada estado?
df_resul

media  mediana   moda  desvio_padrao
estado                                           
PR      648.961294    650.0  850.0      98.607186
RS      651.105143    650.0  850.0      95.136598
SC      649.537653    653.0  850.0      97.233493

In [15]:
# Transpondo o data frame pro formato correto
df_resul = df_resul.T

In [16]:
# Salvando em .json
df_resul.to_json('submission.json')